# Detect Single File
python detect.py --weights .\runs\train\yolov7_Screws4\weights\best.pt --conf 0.8 --img-size 640 --source 14.jpg --no-trace

python .\segment\train.py --data data\custom.yaml --batch 4 --weights yolov7-seg.pt --cfg models\segment\yolov7-seg.yaml --img
 640 --hyp data\hyps\hyp.scratch-high.yaml --epochs 200 --name Final


In [98]:
import argparse
import os
import platform
import sys
from pathlib import Path
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import modules.image_processing as ip
import json

FILE = Path('Custom_YoloV7.ipynb').resolve()
ROOT = FILE.parents[0]  # YOLOv5 root directory

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
   
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from models.common import DetectMultiBackend
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages, LoadStreams
from utils.general import (LOGGER, Profile, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression,scale_segments, print_args, scale_coords, strip_optimizer, xyxy2xywh)
from utils.plots import Annotator, colors, save_one_box
from utils.segment.general import process_mask, scale_masks, masks2segments
from utils.segment.plots import plot_masks
from utils.torch_utils import select_device, smart_inference_mode

import Sorting_Handler as SH


## Setup

In [99]:
model_path = 'runs/train-seg/exp2/weights/best.pt'  
source = '../../../Datasets/Testing/YoloV7/V2'  
imgsz=(640, 640)  # inference size (height, width)
saved_Objects_path='../../../Datasets/Scanned Data/V1'

## Defines

In [108]:
weights=ROOT / model_path  # model.pt path(s)
source=ROOT / source  # file/dir/URL/glob, 0 for webcam
data=ROOT / 'data/coco128.yaml'  # dataset.yaml path
imgsz=(640, 640)  # inference size (height, width)
conf_thres=0.9  # confidence threshold
iou_thres=0.2  # NMS IOU threshold
max_det=1000  # maximum detections per image
device=''  # cuda device, i.e. 0 or 0,1,2,3 or cpu
view_img=False  # show results
save_txt=False  # save results to *.txt
save_conf=False  # save confidences in --save-txt labels
save_crop=False  # save cropped prediction boxes
nosave=False  # do not save images/videos
classes=1  # filter by class: --class 0, or --class 0 2 3
agnostic_nms=False  # class-agnostic NMS
augment=False  # augmented inference
visualize=False  # visualize features
update=False  # update all models
project=ROOT / 'runs/predict-seg'  # save results to project/name
name='exp'  # save results to project/name
exist_ok=False  # existing project/name ok, do not increment
line_thickness=3  # bounding box thickness (pixels)
hide_labels=False  # hide labels
hide_conf=False  # hide confidences
half=False  # use FP16 half-precision inference
dnn=False  # use OpenCV DNN for ONNX inference
trk = False

In [109]:
source = str(source)
is_file = Path(source).suffix[1:] in (IMG_FORMATS + VID_FORMATS)
ROI_Path = source + '\ROI'
  

# Prepare Data
## Clear or Generate ROI folder

In [110]:
if not os.path.exists(ROI_Path):
    os.makedirs(ROI_Path)
else:
    # Delete all existing Files of ROI
    for file in os.listdir(ROI_Path):
        file_path = os.path.join(ROI_Path, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f"Error while deleting file: {file_path}: {e}")

## Cut and Generate ROI

In [111]:
for filename in os.listdir(source):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  
        img_path = os.path.join(source, filename)
        img = cv2.imread(img_path)
            
        # Hier kannst du die Größe anpassen, z.B. 200x200 Pixel
        ROI = img[175:950, 0:1700]  # Beispiel für Zuschneiden auf 200x200 Pixel
            
        output_img_path = os.path.join(ROI_Path, filename)
        cv2.imwrite(output_img_path, ROI)


# Load model

In [112]:
device = select_device(device)
model = DetectMultiBackend( weights, device=device, dnn=dnn, data=data, fp16=half)
stride, names, pt = model.stride, model.names, model.pt
imgsz = check_img_size(imgsz, s=stride)  # check image size

YOLOv5  yolov7-segmentation-37-g9c92765 Python-3.11.5 torch-2.1.2+cpu CPU

Fusing layers... 
yolov7-seg summary: 325 layers, 37847870 parameters, 0 gradients, 141.9 GFLOPs


In [113]:
dataset = LoadImages(ROI_Path, img_size=imgsz, stride=stride, auto=pt)
bs = 1  # batch_size
vid_path, vid_writer = [None] * bs, [None] * bs

# Run inference
model.warmup(imgsz=(1 if pt else bs, 3, *imgsz))  # warmup
seen, windows, dt = 0, [], (Profile(), Profile(), Profile())

In [114]:
img_nr = 0
length_array = []
names_array = []

for path, im, im0s, vid_cap, s in dataset:
    
    original = im0s

    print(original.shape)
    with dt[0]:
        im = torch.from_numpy(im).to(device)
        im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
        im /= 255  # 0 - 255 to 0.0 - 1.0
        if len(im.shape) == 3:
            im = im[None]  # expand for batch dim

    # Inference
    with dt[1]:
        #visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
        pred, out = model(im, augment=augment, visualize=visualize)
        proto = out[1]

    # NMS
    with dt[2]:
        pred = non_max_suppression(pred, 0.2, iou_thres, classes, agnostic_nms, max_det=max_det, nm=32)

    # Second-stage classifier (optional)
    # pred = utils.general.apply_classifier(pred, classifier_model, im, im0s)
    print(f'Prediction in {len(pred)} Image')
    # Process predictions
    for i, det in enumerate(pred):  # per image
        print(f'Detected {len(det)} Objects')
        seen += 1

        p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

        p = Path(p)  # to Path
        #save_path = str(save_dir / p.name)  # im.jpg
        #txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
        s += '%gx%g ' % im.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        imc = im0.copy() if save_crop else im0  # for save_crop
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))
        if len(det):
            masks = process_mask(proto[i], det[:, 6:], det[:, :4], im.shape[2:], upsample=True)  # HWC

            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()

            # Segments
            if save_txt:
                segments = reversed(masks2segments(masks))
                segments = [scale_segments(im.shape[2:], x, im0.shape).round() for x in segments]
            
            # Print results
            for c in det[:, 5].unique():
                n = (det[:, 5] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            # Mask plotting ----------------------------------------------------------------------------------------
            mcolors = [colors(int(cls), True) for cls in det[:, 5]]
            im_masks = plot_masks(im[i], masks, mcolors)  # image with masks shape(imh,imw,3)
            
            object_coordinates = SH.get_object_coordinates_from_mask(masks)
            binary_borders = SH.gen_image(object_coordinates, showImage = False)             
            binary_borders_scaled = scale_masks(im.shape[2:], binary_borders, im0.shape)
            print(f'binary_borders_scaled shape : {binary_borders_scaled.shape}')
            #cv2.imshow('Binary Image (Scaled)', binary_borders_scaled)
            #cv2.waitKey(0)
            
            # Detect the contours in the Threshold Mask 
            contours, hierarchy = cv2.findContours(image=binary_borders_scaled[:,:,0], mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
            print(f'{len(contours)} Contours found')
            
             # Mask plotting ----------------------------------------------------------------------------------------
            annotator.im = scale_masks(im.shape[2:], im_masks, im0.shape)  # scale to original h, w
            # Mask plotting ----------------------------------------------------------------------------------------

           
            # Write results
            for j, (*xyxy, conf, cls) in enumerate(reversed(det[:, :6])):
                if save_txt:  # Write to file
                    segj = segments[j].reshape(-1)  # (n,2) to (n*2)
                    line = (cls, *segj, conf) if save_conf else (cls, *segj)  # label format

                if save_crop:
                    #save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)
                    pass
                
        # Stream results
        im0 = annotator.result()
        
        showContours = False
        if showContours:
            cv2.drawContours(im0, contours, -1, (0, 255, 0), 2)
            cv2.imshow(str(p), im0)
            cv2.waitKey(0)  # 1 millisecond
            cv2.destroyAllWindows()
        
   
    rectangles = ip.get_rects_from_contours(contours)
    bounding_boxes = ip.get_bounding_boxes_from_rectangles(rectangles)
    object_images = ip.warp_objects_horizontal(original, rectangles, bounding_boxes)
        
    standardized = ip.standardize_images(object_images)  
    
    # Generate array with the length of the Objects
    for i,_ in enumerate(object_images):
        length_array.append(max(object_images[i].shape))
        
    # Save Objet to Folder
    for _, image in enumerate(standardized):
        img_nr = img_nr+1
        name = 'Item{}.png'.format(img_nr)
        path = saved_Objects_path +'/'+name
        names_array.append(name)
        cv2.imwrite(path, image)
        
    
for i,_ in enumerate(object_images):
    length_array.append(max(object_images[i][0].shape))

data = {}
for idx, name in enumerate(names_array):
    data[name] = length_array[idx]

# Schreiben der Daten in eine JSON-Datei
with open(os.path.join(saved_Objects_path, 'Dataset.json'), 'w') as file:
    json.dump(data, file)

(775, 1600, 3)
Prediction in 1 Image
Detected 26 Objects
binary_borders_scaled shape : (775, 1600, 3)
28 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 8 Objects
binary_borders_scaled shape : (775, 1600, 3)
9 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 12 Objects
binary_borders_scaled shape : (775, 1600, 3)
14 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 16 Objects
binary_borders_scaled shape : (775, 1600, 3)
13 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 17 Objects
binary_borders_scaled shape : (775, 1600, 3)
15 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 18 Objects
binary_borders_scaled shape : (775, 1600, 3)
19 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 19 Objects
binary_borders_scaled shape : (775, 1600, 3)
16 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detected 20 Objects
binary_borders_scaled shape : (775, 1600, 3)
21 Contours found
(775, 1600, 3)
Prediction in 1 Image
Detec